In [6]:
import sys, platform

base_path = '/home/yurid/git/tcc'

if platform.system() == 'Darwin':
    base_path = '/Users/yuri/git/diasYuri/tcc/radiation-forecast-mg'
sys.path.insert(0, base_path)

In [7]:
import os
import pandas as pd
from src.dataframe import read_data, read_data_from_csv
import src.filler as fl
import numpy as np


2023-11-20 15:16:54.374690: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-20 15:16:57.937971: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [59]:
directory = f'{base_path}/data'
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    if os.path.isfile(f):
        df = read_data(f)
        print(f.split('/')[-1:][0], 'dados faltantes:', '\n', df.isna().sum())

daily_barbacena.csv dados faltantes: 
 RADIATION     273
TEMP          326
HUMIDITY_h    667
dtype: int64
daily_juiz de fora.csv dados faltantes: 
 RADIATION     34
TEMP          34
HUMIDITY_h    34
dtype: int64
daily_vicosa.csv dados faltantes: 
 RADIATION     152
TEMP           92
HUMIDITY_h     94
dtype: int64
daily_sao joao del rei.csv dados faltantes: 
 RADIATION     137
TEMP          137
HUMIDITY_h    138
dtype: int64
daily_muriae.csv dados faltantes: 
 RADIATION     131
TEMP          109
HUMIDITY_h    109
dtype: int64


In [5]:
directory = f'{base_path}/data'
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    if os.path.isfile(f):
        df = read_data_from_csv(f).RADIATION
        df_complete = fl.FillerHelper.get_largest_complete_interval(df)
        print(f.split('/')[-1:][0], 'Maior lacuna de dados completos:', len(df_complete))

daily_barbacena.csv Maior lacuna de dados completos: 3126
daily_juiz de fora.csv Maior lacuna de dados completos: 3221
daily_vicosa.csv Maior lacuna de dados completos: 3839
daily_sao joao del rei.csv Maior lacuna de dados completos: 1509
daily_muriae.csv Maior lacuna de dados completos: 1841


In [82]:
def read_data_2(filename):
    df = pd.read_csv(
        filename,
        sep=',',
        parse_dates=['Timestamp'],
        index_col=['Timestamp'])

    df = df[['RADIATION', 'TEMP', 'HUMIDITY_h']]

    dict_index = {}
    df_temp = df.copy().resample('D').mean()
    for c in df_temp:
        dict_index[c] = df_temp[c].loc[np.isnan(df_temp[c].values)].index
    
    df = df.interpolate()\
        .resample('D') \
        .agg({'RADIATION': np.sum, 'TEMP': np.mean, 'HUMIDITY_h': np.mean})\
        .replace(0, np.nan)

    for c in df:
        df[c].loc[dict_index[c]] = pd.NA

    return df.loc[df.index >= df[~df.RADIATION.isna()].index[0]]

In [83]:
df = read_data_2(f'{base_path}/data/daily_barbacena.csv')
df2 = read_data(f'{base_path}/data/daily_barbacena.csv')

len(df), len(df2)

/var/folders/tf/7zgm_5115vjc9x3162_qjg140000gn/T/ipykernel_33362/2527762277.py:15: FutureWarning: The provided callable <function sum at 0x10c546820> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  df = df.interpolate()\
/var/folders/tf/7zgm_5115vjc9x3162_qjg140000gn/T/ipykernel_33362/2527762277.py:15: FutureWarning: The provided callable <function mean at 0x10c54aca0> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df = df.interpolate()\


(7226, 7226)

In [87]:
diff = df.compare(df2)


diff

RADIATION          TEMP       HUMIDITY_h      
                    self    other self other       self other
Timestamp                                                    
2003-03-21  17454.923077  17306.0  NaN   NaN        NaN   NaN
2003-03-22  11003.713287  10357.0  NaN   NaN        NaN   NaN
2003-03-23  11044.286713  10708.0  NaN   NaN        NaN   NaN
2003-03-24  13792.826923  12355.0  NaN   NaN        NaN   NaN
2003-03-25  20811.596154  19740.0  NaN   NaN        NaN   NaN
...                  ...      ...  ...   ...        ...   ...
2022-12-27  14148.363636  13899.2  NaN   NaN        NaN   NaN
2022-12-28  10504.739394  10267.3  NaN   NaN        NaN   NaN
2022-12-29   8219.560606   8122.0  NaN   NaN        NaN   NaN
2022-12-30   8244.227273   7987.6  NaN   NaN        NaN   NaN
2022-12-31  12598.745455  12501.4  NaN   NaN        NaN   NaN

[5298 rows x 6 columns]

In [13]:
df = read_data(f'{base_path}/data/daily_vicosa.csv')

df_len = len(df)
train, validation, teste = df[:int(0.7*df_len)], df[int(0.7*df_len):int(0.8*df_len)], df[int(0.8*df_len):]

print(train.index[0], train.index[-1])
print(validation.index[0], validation.index[-1])
print(teste.index[0], teste.index[-1])

2005-09-15 00:00:00 2017-10-22 00:00:00
2017-10-23 00:00:00 2019-07-16 00:00:00
2019-07-17 00:00:00 2022-12-31 00:00:00
